# The PolInSAR Course - June 17, 2024
# Polarimetric SAR Interferometry (Pol-InSAR) 
# Part 2: Pol-InSAR forest height inversion 

* DLR's F-SAR acquisition over Traunstein forest (Germany)
* Path: '/projects/data/04-polinsar/'
* SLCs: 
    * Acquisition 1 : slc_15tmpsar0302_L { hh, hv, vv, vh } _t01.rat
    * Acquisition 2 : slc_coreg_15tmpsar0302_15tmpsar0303_L { hh, hv, vv, vh } _t01.rat
* Flat-earth: pha_flat_15tmpsar0302_15tmpsar0303_Lhh_t01.rat
* Vertical wavenumber (kz) : kz_2d_demc_15tmpsar0302_15tmpsar0303_t01.rat
* Lidar: Lida_r1503.rat

Objective:
- Estimate forest height using the phase extremes of the dual-pol Pol-InSAR coherence region and model inversion.

Tips:
- work on the azimuth - range block [21500 - 4000, 21500 + 6000] - [2300, 4500] ;
- all the needed functions and a few pieces of script have been already implemented.

In [ ]:
# --- Download exercise data & import reader function

from pysarpro import io, data
from pysarpro.io import rrat

# data.download_all(directory="/projects", pattern=r'^data/polinsar')

# ---

In [ ]:
# import useful libraries, functions, and modules

import sys

sys.path.append('/projects/src/')

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import filters

from tqdm import tqdm

%matplotlib widget

**Auxiliary functions**

`calculate_covariance`: Calculates the covariance between two images while performing a multi-looking operation.

In [ ]:
def calculate_covariance(im1, im2, looksr, looksa) : 
    
    # ... apply definition
    corr = filters.uniform_filter(np.real(im1*np.conj(im2)), [looksa,looksr]) + 1j* \
                filters.uniform_filter(np.imag(im1*np.conj(im2)), [looksa,looksr])
    
    # ... and back to main
    return corr

`make_pauli`: Generates an RGB Pauli image given the SLCs of the different polarimetric channels.

In [ ]:
def make_pauli(slchh, slchv, slcvv, looksr, looksa) :
    
    # 1. Uses function calculate_covariance
    # 2. Convention (rows x columns): inputs are (az x rg), outputs are (rg x az) 
    #                                 for better plotting as #rg <#az
    
    # Calculate T11, T22 and T33
    T11 = calculate_covariance(slchh + slcvv, slchh + slcvv, looksr, looksa)
    T22 = calculate_covariance(slchh - slcvv, slchh - slcvv, looksr, looksa)
    T33 = calculate_covariance(2*slchv, 2*slchv, looksr, looksa)
    
    # make the pauli rgb (+ tranpose, clipping, normalization)
    dimaz = T11.shape[0]
    dimrg = T11.shape[1]
    rgb_pauli = np.zeros((dimrg, dimaz, 3), 'float32')
    rgb_pauli[:, :, 0] = np.transpose( np.clip(np.sqrt(np.abs(T22)), 0, 2.5*np.mean(np.sqrt(np.abs(T22)))) )    # red
    rgb_pauli[:, :, 1] = np.transpose( np.clip(np.sqrt(np.abs(T33)), 0, 2.5*np.mean(np.sqrt(np.abs(T33)))) )    # green
    rgb_pauli[:, :, 2] = np.transpose( np.clip(np.sqrt(np.abs(T11)), 0, 2.5*np.mean(np.sqrt(np.abs(T11)))) )    # blue
    rgb_pauli[:, :, 0] = rgb_pauli[:, :, 0] / np.max(rgb_pauli[:, :, 0])     # red
    rgb_pauli[:, :, 1] = rgb_pauli[:, :, 1] / np.max(rgb_pauli[:, :, 1])     # green
    rgb_pauli[:, :, 2] = rgb_pauli[:, :, 2] / np.max(rgb_pauli[:, :, 2])     # blue
    
    # ... and back to main
    return rgb_pauli

`matmul_2`: Mutiplies two matrices given the elements of each matrix separately.

In [ ]:
def matmul_2(a11, a12, a21, a22, b11, b12, b21, b22) :
    
    # Implements the multiplication A.B of two 2 x 2 matrices A and B.
    # Inputs are the elements of matrices (row-wise). 
    # Outputs are the elements of the matrix product (row-wise).
    
    c11 = a11*b11 + a12*b21
    c12 = a11*b12 + a12*b22
    c21 = a21*b11 + a22*b21 
    c22 = a21*b12 + a22*b22
    
    # ... and back to main
    return c11, c12, c21, c22

`sqrt_inverse`: Calculates the square-root inverse of a 2x2 Hermitian matrix. **See Appendix 1.**

In [ ]:
def sqrt_inverse(a11, a12, a22) :

    # Given a 2 x 2 Hermitian matrix A, calculates A**(-1/2) using eigendecomposition
    # Inputs are the upper-diagonal matrix elements (row-wise) - NumPy arrays.
    # Outputs are the elements of the sqrt inverse matrix.
    # Uses function matmul_2.
    
    # Calculate eigenvalues (l1, l2)
    sigma = 0.5 * (a11 + a22)
    D = a11*a22 - a12*np.conj(a12)
    delta = np.sqrt(sigma**2 - D)
    l1 = sigma + delta
    l2 = sigma - delta
    
    # Calculate eigenvector matrix elements (eigenvectors are the columns)
    norm_ = np.sqrt( a12*np.conj(a12) + ((a11-a22)/2 - delta)**2 )
    v11 = -a12 / norm_
    v21 = ((a11-a22)/2 - delta) / norm_
    v12 = ((a11-a22)/2 - delta) / norm_
    v22 = np.conj(a12) / norm_
    
    del norm_, delta, D, sigma
    
    # calculate sqrt inverse matrix elements
    zz = np.zeros(l1.shape, 'float32')
    aux11, aux12, aux21, aux22 = matmul_2(v11, v12, v21, v22, 1/np.sqrt(l1), zz, zz, 1/np.sqrt(l2))
    del l1, l2
    i11, i12, i21, i22 = matmul_2(aux11, aux12, aux21, aux22, np.conj(v11), np.conj(v21), np.conj(v12), np.conj(v22))
    del aux11, aux12, aux21, aux22
    
    # ... and back to main
    return i11, i12, i21, i22

`eigenvalvect_2`: Computes the eigenvalues and eigenvectors of a 2x2 matrix analytically. **See Appendix 2.**

In [ ]:
def eigenvalvect_2(a11, a12, a21, a22) :
    
    # Calculates eigenvalues / eigenvectors of a generic complex-valued 2 x 2 matrix.
    # Inputs are the matrix elements (row-wise) - NumPy arrays
    # Outputs are the eigenvalues (l1, l2), and the eigenvector (E1, E2) elements - NumPy arrays
    #                                       E1 = [e11, e21] , U2 = [e12, e22]
    
    # eigenvalues 
    l1 = 0.5 * ((a11+a22) + np.sqrt( (a11-a22)**2 + 4*a12*a21 )) 
    l2 = 0.5 * ((a11+a22) - np.sqrt( (a11-a22)**2 + 4*a12*a21 ))
    
    # eigenvector 1
    e11 = -a12
    e21 = a11 - l1
    # normalize eigenvector 1
    norm_ = np.sqrt( np.abs(e11)**2 + np.abs(e21)**2 )
    e11 = e11 / norm_
    e21 = e21 / norm_
    # eigenvector 2
    e12 = a22 - l2
    e22 = -a21
    # normalize eigenvector 2
    norm_ = np.sqrt( np.abs(e12)**2 + np.abs(e22)**2 )
    e12 = e12 / norm_
    e22 = e22 / norm_
    
    del norm_
    
    # ... and back to main
    return l1, l2, e11, e21, e12, e22

`plot_coherence_region_P`: Reconstructs the **boundary** of the **Coherence Region**.

<div>
<img src="img/01_boundary_core.png" width="800"/>
</div>

In [ ]:
def plot_coherence_region_P(P11, P12, P21, P22, npoints=128, axes=None, color='r', **kwargs):
    
    # Pre-whitened coherency matrix
    M = np.asarray([[P11, P12], [P21, P22]])
    
    # Phasors
    theta = np.linspace(0, np.pi, npoints)
    
    # Optimizatin problem as an eigenvalue equation
    Wi,Vi = np.linalg.eigh(0.5*(M[None,...]*np.exp(1j*theta)[...,None,None] + np.conj(M.T)[None,...]*np.exp(-1j*theta)[...,None,None]))
    
    # Axes in polar coordinates
    if axes is None: ax = plt.subplot(111, projection='polar')
    else: ax = axes
    
    # Reconstruct the border of the Coherence Region
    indices = np.argsort(Wi, axis=-1)
    Z = np.einsum('...i,...ij,...j->...', np.conj(Vi[range(npoints),:,indices[:,-1]]), M, Vi[range(npoints),:,indices[:,-1]])
    Z2 = np.einsum('...i,...ij,...j->...', np.conj(Vi[range(npoints),:,indices[:,0]]), M, Vi[range(npoints),:,indices[:,0]])
    ax.plot(np.angle(Z), np.abs(Z), color, **kwargs)
    ax.plot(np.angle(Z2), np.abs(Z2), color, **kwargs)
    
    # ... and back to main
    return ax

`optimize_coherence_and_phase`: By using previous auxiliary functions, this function performs a **Polar Decomposition** of a 2x2 matrix. As a result, it provides the **optimized extreme coherences**.

<div>
<img src="img/03_polar_decomp.PNG" width="700"/>
</div>

In [ ]:
def optimize_coherence_and_phase(P11, P12, P21, P22):
    """
    Performs the Polar decomposition of the 2 by 2 matrix P with the given elements
    and estimates coherences with min max phase and absolute value.
    
    The polar decomposition of P is P = U * J.
    The max & min coherence is extracted from the eigenvectors of J and the
    max & min phase from the eigenvectors of U
    
    See: https://en.wikipedia.org/wiki/Polar_decomposition
    """
    
    # Calculate J^2 = P^(*T) * P
    J11, J12, J21, J22 = matmul_2(np.conj(P11), np.conj(P21), np.conj(P12), np.conj(P22), P11, P12, P21, P22)
    
    # Calculate eigenvalues and eigenvectors of J
    l1, l2, v11, v21, v12, v22 = eigenvalvect_2(J11, J12, J21, J22)
    del l1, l2
    
    # Calculate V^(*T) * P * V --> gamma max & min from there
    aux11, aux12, aux21, aux22 = matmul_2(np.conj(v11), np.conj(v21), np.conj(v12), np.conj(v22), P11, P12, P21, P22)
    gamma_max, gamma12, gamma21, gamma_min = matmul_2(aux11, aux12, aux21, aux22, v11, v12, v21, v22)
    del aux11, aux12, aux21, aux22, v11, v21, v12, v22
    del gamma12, gamma21
    
    # Compute U = P * J^(-1)
    Ji11, Ji1, Ji21, Ji22 = sqrt_inverse(J11, J12, J22)
    del J11, J12, J21, J22
    U11, U12, U21, U22 = matmul_2(P11, P12, P21, P22, Ji11, Ji1, Ji21, Ji22)
    del Ji11, Ji1, Ji21, Ji22
    
    # Calculate eigenvalues and eigenvectors of U
    l1, l2, w11, w21, w12, w22 = eigenvalvect_2(U11, U12, U21, U22)
    del l1, l2, U11, U12, U21, U22
    
    # Calculate W^(*T) * P * W --> gamma phamax & phamin from there
    aux11, aux12, aux21, aux22 = matmul_2(np.conj(w11), np.conj(w21), np.conj(w12), np.conj(w22), P11, P12, P21, P22)
    gamma_phamax, gamma12, gamma21, gamma_phamin = matmul_2(aux11, aux12, aux21, aux22, w11, w12, w21, w22)
    del aux11, aux12, aux21, aux22, w11, w21, w12, w22
    del gamma12, gamma21
    
    # Return optimum coherences
    return gamma_phamax, gamma_phamin, gamma_max, gamma_min

**Input parameters**

In [ ]:
# --- Inputs

# path 2 acquisitions
path = '/projects/data/polinsar/'

# input pixel spacing, in meters
spacrg = 0.59941552
spacaz = 0.19507939

# output range resolution, in meters
resrg = 5.
resaz = 5.

# image block for processing
minrg = 2300
maxrg = 4500
minaz = 21500 - 4000
maxaz = 21500 + 6000

In [ ]:
# --- Calculate number of looks

looksr = int( resrg / spacrg )
if looksr % 2 == 0 : looksr = looksr +1
looksa = int( resaz / spacaz )
if looksa % 2 == 0 : looksa = looksa +1

**Step 1: Open images, and visualize a Pauli**

In [ ]:
# --- Acquisition 1
slchh1 = rrat(path + 'slc_15tmpsar0302_Lhh_t01.rat', block=[minaz, maxaz, minrg, maxrg])
slchv1 = rrat(path + 'slc_15tmpsar0302_Lhv_t01.rat', block=[minaz, maxaz, minrg, maxrg])
slcvv1 = rrat(path + 'slc_15tmpsar0302_Lvv_t01.rat', block=[minaz, maxaz, minrg, maxrg])

# --- Acquisition 2
slchh2 = rrat(path + 'slc_coreg_15tmpsar0302_15tmpsar0303_Lhh_t01.rat', block=[minaz, maxaz, minrg, maxrg])
slchv2 = rrat(path + 'slc_coreg_15tmpsar0302_15tmpsar0303_Lhv_t01.rat', block=[minaz, maxaz, minrg, maxrg])
slcvv2 = rrat(path + 'slc_coreg_15tmpsar0302_15tmpsar0303_Lvv_t01.rat', block=[minaz, maxaz, minrg, maxrg])

In [ ]:
# --- Pauli RGB
pauli_rgb = make_pauli(slchh1, slchv1, slcvv1, looksr, looksa)

# display a Pauli
plt.figure(figsize=(10,4))
plt.imshow(pauli_rgb, aspect='auto')
plt.tight_layout()

# free memory
del pauli_rgb

**Step 2 : Compensate flat-earth**

In [ ]:
# --- Open flat-earth
fe = rrat(path + 'pha_flat_15tmpsar0302_15tmpsar0303_Lhh_t01.rat', block = [minaz, maxaz, minrg, maxrg])

# compensate
slchh2 = slchh2 * np.exp(1j*fe)
slchv2 = slchv2 * np.exp(1j*fe)
slcvv2 = slcvv2 * np.exp(1j*fe)

# free memory
del fe

**Step 3 : Calculate the elements of T1, T2, and Omega matrices**

<div>
<img src="img/04_PolInSAR_matrices.png" width="400"/>
</div>


In [ ]:
# --- Form polarization channels
# acquistion 1
pol1_im1 = slchh1 - slcvv1
pol2_im1 = 2*slchv1
# acquisition 2
pol1_im2 = slchh2 - slcvv2
pol2_im2 = 2*slchv2

# free memory 
del slchh1, slchv1, slcvv1, slchh2, slchv2, slcvv2

# --- T1
T1_11 = calculate_covariance(pol1_im1, pol1_im1, looksr, looksa)
T1_12 = calculate_covariance(pol1_im1, pol2_im1, looksr, looksa)
T1_22 = calculate_covariance(pol2_im1, pol2_im1, looksr, looksa)

# --- T2
T2_11 = calculate_covariance(pol1_im2, pol1_im2, looksr, looksa)
T2_12 = calculate_covariance(pol1_im2, pol2_im2, looksr, looksa)
T2_22 = calculate_covariance(pol2_im2, pol2_im2, looksr, looksa)

# --- Omega  
Om_11 = calculate_covariance(pol1_im1, pol1_im2, looksr, looksa)
Om_12 = calculate_covariance(pol1_im1, pol2_im2, looksr, looksa)
Om_21 = calculate_covariance(pol2_im1, pol1_im2, looksr, looksa)
Om_22 = calculate_covariance(pol2_im1, pol2_im2, looksr, looksa)

# free memory
del pol1_im1, pol2_im1, pol1_im2, pol2_im2

**Step 4 : Pre-whitening (normalization)**

<div>
<img src="img/05_pre-whitening_2.PNG" width="450"/>
</div>

In [ ]:
# --- Pre-whitening

# calculate T
T11 = 0.5 * ( T1_11 + T2_11 )
T12 = 0.5 * ( T1_12 + T2_12 )
T22 = 0.5 * ( T1_22 + T2_22 )

# free memory
del T1_11, T1_12, T1_22, T2_11, T2_12, T2_22

# calculate the elements of T**(-1/2)
iT11, iT12, iT21, iT22 = sqrt_inverse(T11, T12, T22)

# free memory
del T11, T12, T22

# whiten ... two 2 x 2 multiplications ==> Matrix P
aux11, aux12, aux21, aux22 = matmul_2(iT11, iT12, iT21, iT22, Om_11, Om_12, Om_21, Om_22)
P11, P12, P21, P22 = matmul_2(aux11, aux12, aux21, aux22, iT11, iT12, iT21, iT22)

# free memory
del aux11, aux12, aux21, aux22
del iT11, iT12, iT21, iT22

**Step 5: Get optimized coherences with minimum/maximum phase**

<div>
<img src="img/03_polar_decomp.PNG" width="700"/>
</div>

In [ ]:
# --- Optimized coherences in terms of phase and amplitude 
gamma_phamax, gamma_phamin, gamma_max, gamma_min = optimize_coherence_and_phase(P11, P12, P21, P22)

# free memory 
del gamma_max, gamma_min

**Step 6: Find the volume-only coherence**

In [ ]:
# --- Open vertical wavenumber
kz = rrat(path + 'kz_2d_demc_15tmpsar0302_15tmpsar0303_t01.rat', block=[minaz, maxaz, minrg, maxrg])

In [ ]:
# display kz
plt.figure(figsize=(4,9))
plt.imshow(kz, cmap='jet', vmin=0, vmax=0.25, aspect='auto')
cb = plt.colorbar(orientation='horizontal', pad=0.03)
cb.set_label('[rad/m]')
plt.title('kz')
plt.tight_layout()

In [ ]:
# Test if height(gamma_phamax) > height(gamma_phamin) 

height_diff = np.angle( gamma_phamax*np.conj(gamma_phamin) ) /kz

plt.figure(figsize=(4,9))
plt.imshow(height_diff, cmap='jet', vmin=-10, vmax=10, aspect='auto')
cb = plt.colorbar(orientation='horizontal', pad=0.03)
cb.set_label('[m]')
plt.title('Phase center height difference')
plt.tight_layout()


In [ ]:
# --- Order phase values  

ii_az, ii_rg = np.where(height_diff < 0)
if ii_az.size >0 :
    gamma_phamax[ii_az, ii_rg], gamma_phamin[ii_az, ii_rg] = gamma_phamin[ii_az, ii_rg], gamma_phamax[ii_az, ii_rg]

# --- Decide gamma_v ... assuming mu_min = 0  

gamma_v = gamma_phamax


In [ ]:
# Test (again) if height(gamma_phamax) > height(gamma_phamin)  

plt.figure(figsize=(4,9))
plt.imshow(np.angle( gamma_phamax*np.conj(gamma_phamin) ) /kz, cmap='jet', vmin=-10, vmax=10, aspect='auto')
cb = plt.colorbar(orientation='horizontal', pad=0.03)
cb.set_label('[m]')
plt.title('Phase center height difference after re-ordering')
plt.tight_layout()


**Step 7: Calculate ground phase**

<div>
<img src="img/08_ground_phase_eq.png" width="600"/>
</div>

In [ ]:
# --- Calculate ground phase
v = gamma_phamin - gamma_v
x = -np.real( np.conj(v)*gamma_v ) / np.abs(v)**2 + \
            np.sqrt( np.real(np.conj(v)*gamma_v)**2 - (np.abs(gamma_v)**2 - 1)* np.abs(v)**2 ) / np.abs(v)**2
gamma_g = gamma_v + v*x 
phi0  = np.angle(gamma_g)

In [ ]:
# --- display: phase of gamma_v, gamma_phamin, phi0

plt.figure(figsize=(9,10))

plt.subplot(131) 
plt.imshow(np.angle(gamma_v), vmin=-np.pi, vmax=np.pi, cmap='jet', aspect='auto')
cb = plt.colorbar(orientation='horizontal', pad=0.03)
cb.set_label('[rad]')
plt.title('Phase gamma_v')
plt.axis('off')

plt.subplot(132) 
plt.imshow(np.angle(gamma_phamin), vmin=-np.pi, vmax=np.pi, cmap='jet', aspect='auto')
cb = plt.colorbar(orientation='horizontal', pad=0.03)
cb.set_label('[rad]')
plt.title('Phase gamma_phamin')
plt.axis('off')

plt.subplot(133) 
plt.imshow(phi0, vmin=-np.pi, vmax=np.pi, cmap='jet', aspect='auto')
cb = plt.colorbar(orientation='horizontal', pad=0.03)
cb.set_label('[rad]')
plt.title('Ground phase')
plt.axis('off')

plt.tight_layout()

**Step 8: Visualize the Coherence Region for one pixel**

In [ ]:
# --- Plot coherence region for one pixel
px_az = 6000
px_rg = 500

# --- Get the Pi matrix for the selected pixel
px_P11 = P11[px_az, px_rg]
px_P12 = P12[px_az, px_rg]
px_P21 = P21[px_az, px_rg]
px_P22 = P22[px_az, px_rg]
# --- Get the coherences for the selected pixel
px_gv = gamma_v[px_az, px_rg]
px_gphamin = gamma_phamin[px_az, px_rg]
px_gg = gamma_g[px_az, px_rg]

# --- Plot Coherence Region 
plt.figure()

# BEFORE compensating phi0
ax = plot_coherence_region_P(px_P11, px_P12, px_P21, px_P22, color='k')
ax.scatter(np.angle(px_gv), abs(px_gv), c='g')
ax.scatter(np.angle(px_gphamin), abs(px_gphamin), c='r')

# calculate Pol-InSAR line parameters 
slope = np.imag(px_gv - px_gphamin) / np.real(px_gv - px_gphamin)
qq = np.imag(px_gv) - slope * np.real(px_gv)
# now the line for 2 points
xx = np.asarray([-1, 1])
yy = slope*xx + qq
points = xx + 1j*yy
# and now plot
ax.plot(np.angle(points), np.abs(points), c = 'k')

ax.scatter(np.angle(px_gg), abs(px_gg), c='brown')

# --- Compensate phi0 ... 
px_P11 = P11[px_az, px_rg] * np.conj(px_gg)
px_P12 = P12[px_az, px_rg] * np.conj(px_gg)
px_P21 = P21[px_az, px_rg] * np.conj(px_gg)
px_P22 = P22[px_az, px_rg] * np.conj(px_gg)
# ... Over coh 
px_gv = gamma_v[px_az, px_rg] * np.conj(px_gg)
px_gphamin = gamma_phamin[px_az, px_rg] * np.conj(px_gg)
px_gg = gamma_g[px_az, px_rg] * np.conj(px_gg)
# plot AFTER compensating phi0  <======= TO BE COMPLETED !
ax = plot_coherence_region_P(px_P11, px_P12, px_P21, px_P22, color='b')
ax.scatter(np.angle(px_gv), abs(px_gv), c='g')
ax.scatter(np.angle(px_gphamin), abs(px_gphamin), c='r')
ax.plot(np.angle([px_gg, px_gv]), np.abs([px_gg, px_gv]), c='b')

# Coherence Region normalized to the Unit Circle: from 0 to 1
ax.set_rmax(1)


**Step 9: Compensate ground phase**

In [ ]:
# --- Volume-only coherence  <======= TO BE COMPLETED !


**Step 10: Plot look-up table for the chosen pixel**

<div>
<img src="img/09_volume_coherence_model_eq.png" width="600"/>
</div>

In [ ]:
# --- Open incidence angle
incang = rrat(path + 'incidence_15tmpsar0302_L_t01.rat', block=[minaz, maxaz, minrg, maxrg])

In [ ]:
# define interval of variation of height and extinction  <======= TO BE COMPLETED !


# --- Calculate LUT = Look-up table = all the coherences for every combination of height and extinction <======= TO BE COMPLETED !


**Step 11: Forest height inversion & validation**

In [ ]:
# --- Estimate forest height  

# Get positions in rg and az to process
pos_rg = np.arange(0, gamma_v0.shape[1], looksr)
pos_az = np.arange(0, gamma_v0.shape[0], looksa)

# Make meshgrid
rgm, azm = np.meshgrid(pos_rg, pos_az)

# Flatten indexes 
rgm = rgm.flatten()
azm = azm.flatten()

# --- Define interval of variation of height and extinction <======= TO BE COMPLETED !


# Make meshgrid H, s <======= TO BE COMPLETED !


# Define output arrays 
Hest = np.zeros(rgm.shape)
sest = np.zeros(rgm.shape)

for i in tqdm( range(rgm.size) ):  # <======= TO BE COMPLETED !

    
# Reshape Hest
Hest = np.reshape(Hest, (pos_az.size, pos_rg.size))

In [ ]:
# --- Open lidar height
H = rrat(path + 'Lida_r1503.rat', block=[minaz, maxaz, minrg, maxrg])

In [ ]:
# display estimates  

Hl_small = H[pos_az,:]
Hl_small = Hl_small[:,pos_rg]

kz_small = kz[pos_az,:]
kz_small = kz_small[:,pos_rg]

gamma_v0_s = gamma_v0[pos_az,:]
gamma_v0_s = gamma_v0_s[:,pos_rg]

# Mask out non relevant estimates  <======= TO BE COMPLETED !

# display 
plt.figure(figsize=(6,10))

plt.subplot(1,2,1)
plt.imshow(Hest, vmin=0, vmax=50, cmap='jet', aspect='auto')
plt.colorbar(orientation='horizontal', pad=0.05)
plt.title('PolInSAR height (m)')

plt.subplot(1,2,2)
plt.imshow(Hl_small, vmin=0, vmax=50, cmap='jet', aspect='auto')
plt.colorbar(orientation='horizontal', pad=0.05)
plt.title('Lidar height (m)')

plt.tight_layout()

In [ ]:
# --- Validation  

# 2D histogram Lidar height vs PolInSAR height  <======= TO BE COMPLETED !
hist2d = 

# display
plt.figure(figsize=(6,6))
plt.imshow(np.transpose(hist2d), cmap='jet', origin='lower', aspect='auto', extent=[0, 50, 0, 50])
plt.plot([0, 50], [0, 50], "w--")
plt.xlabel('Lidar height (m)')
plt.ylabel('PolInSAR height (m)')
plt.grid()
plt.tight_layout()